<a href="https://colab.research.google.com/github/katerinavr/ml-tutes/blob/master/deep_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/lukasruff/Deep-SVDD-PyTorch.git

Cloning into 'Deep-SVDD-PyTorch'...
remote: Enumerating objects: 488, done.
remote: Total 488 (delta 0), reused 0 (delta 0), pack-reused 488
Receiving objects: 100% (488/488), 2.15 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (320/320), done.


In [0]:
%cd Deep-SVDD-PyTorch

/content/Deep-SVDD-PyTorch/src/Deep-SVDD-PyTorch


In [0]:
%cd src

/content/Deep-SVDD-PyTorch/src/Deep-SVDD-PyTorch/src


In [0]:
!pip install -r requirements.txt


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [0]:
!python main.py mnist mnist_LeNet ../log/mnist_test ../data --objective one-class --lr 0.0001 --n_epochs 150 --lr_milestone 50 --batch_size 200 --weight_decay 0.5e-6 --pretrain True --ae_lr 0.0001 --ae_n_epochs 150 --ae_lr_milestone 50 --ae_batch_size 200 --ae_weight_decay 0.5e-3 --normal_class 3;

Usage: main.py [OPTIONS] [mnist|cifar10] [mnist_Le
               Net|cifar10_LeNet|cifar10_LeNet_ELU
               ] XP_PATH DATA_PATH
Try "main.py --help" for help.

Error: Invalid value for "XP_PATH": Path "../log/mnist_test" does not exist.


In [0]:
from sklearn import datasets, metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'https://drive.google.com/open?id=10TDVemIVsZYCtdEQsF4TVdYpZOeaEnAl'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('colab_test.csv')  
data = pd.read_csv('colab_test.csv')
data.head()

10TDVemIVsZYCtdEQsF4TVdYpZOeaEnAl


,NAME,MW,ZM1V,DBI,Xt,MSD,ICR,PW5,MWC01,TPC,X3A,IDDE,Xindex,Yindex,CIC0,SpMax_A,VE1_A,VE2_D,VE1_X,VE2_D/Dt,SM1_Dz(Z),WiA_Dz(v),Chi_Dz(v),WiA_Dz(p),HyWi_B(m),EE_B(m),VE1_B(m),VE1_B(v),SpDiam_B(p),HyWi_B(s),EE_B(s),GATS7m,GATS8m,GATS7v,GATS8v,GATS7e,GATS7p,GATS8p,GATS7i,GATS8i,GATS7s,GGI2,JGI5,JGI7,JGI8,SpMax4_Bh(m),SpMax5_Bh(m),SpMax2_Bh(v),SpMax4_Bh(v),SpMax2_Bh(p),SpMax5_Bh(s),SpMax7_Bh(s),SpMin2_Bh(m),SpMin3_Bh(m),SpMin3_Bh(v),SpMin2_Bh(s),SpMaxA_EA(ed),SpMax_AEA(dm),SM02_AEA(dm),SM11_AEA(dm),SM12_AEA(dm),Eig02_AEA(dm),Eig08_AEA(dm),Ui,DLS_01,LLS_01,label
0,lab1,839.800000,478.000000,6.164000,-0.291000,3.869000,1.474000,0.004000,1.946000,5.091000,-0.11300,2.193000,-0.432000,-1.030000,2.676000,0.6460,3.704000,-0.087896,4.02300,-0.113422,2.464000,2.088000,-0.63500,1.223000,2.303000,2.784000,4.15400,3.87800,1.038000,1.913000,1.759000,1.149000,0.802000,1.012000,0.927000,1.572000,0.801000,1.043000,0.973000,0.954000,2.043000,9.333000,0.077000,0.040000,0.008000,3.528000,2.987000,1.481000,2.82100,2.279000,2.409000,3.361000,0.537000,0.521000,0.629000,1.391000,-0.328000,1.935000,2.899000,13.969000,13.591000,2.855000,2.89900,-2.00000,-0.250000,-0.500000,1
1,lab2,105.600000,204.000000,0.000000,0.117000,-0.484000,-0.579000,-0.066000,-0.431000,-2.131000,0.05000,-2.125000,0.383000,1.002000,-1.613000,-0.1550,-0.171000,-0.002680,-0.03100,-0.005946,1.384000,0.208000,-0.12000,0.976000,0.637000,2.471000,-0.38800,-0.18500,-0.269000,0.962000,3.599000,-1.016000,0.000000,-1.016000,0.000000,-1.016000,-1.016000,0.000000,-1.016000,0.000000,-1.219000,0.889000,-0.016000,0.000000,0.000000,0.023000,-0.147000,-0.739000,-1.35500,-0.641000,0.718000,-0.339000,-1.247000,-1.044000,-0.676000,-1.204000,0.251000,0.367000,-2.150000,-1.322000,-0.041000,0.448000,-1.34300,-1.39200,0.000000,-0.160000,1
2,lab3,1686.090000,2010.000000,7.895000,-0.359000,9.659000,2.327000,0.038000,2.811000,8.387000,-0.06200,1.312000,-0.712000,-1.723000,3.131000,0.4560,5.727000,-0.120680,8.10300,-0.153946,2.485000,8.492000,-0.74100,9.553000,2.986000,3.352000,4.25900,6.85300,0.961000,3.415000,5.231000,1.091000,0.956000,0.927000,0.933000,1.518000,0.975000,0.979000,0.942000,0.994000,1.445000,20.889000,0.036000,0.018000,0.012000,2.535000,3.679000,0.986000,1.71700,1.029000,5.726000,6.529000,0.482000,0.854000,0.982000,0.681000,-0.780000,2.678000,3.386000,9.651000,10.434000,3.754000,5.24800,3.72800,-0.750000,-0.830000,1
3,lab4,-16.145978,-118.515379,1.481367,-0.076122,0.183183,0.506108,0.047077,0.307804,0.031878,-0.03947,0.038107,0.091082,0.281758,1.261976,0.2263,0.219062,-0.012576,0.48875,-0.017368,-0.288269,0.642088,-0.33976,0.408453,0.224018,0.074431,0.11444,0.20434,0.316515,0.106187,-0.321295,0.267941,0.000302,0.310088,0.031889,0.254907,0.276119,0.003769,0.274116,-0.003564,0.470759,0.561294,0.022727,-0.003638,-0.002659,0.825225,1.084745,0.231676,1.00226,0.168257,0.808882,1.429276,0.300853,0.416853,0.424089,0.404697,0.063067,0.686208,-0.260492,2.454443,3.294731,0.673106,-0.34207,-0.71864,-0.111236,-0.153835,1
4,lab5,-1394.090000,-1186.000000,-7.660000,0.355000,-8.077000,-1.788000,-0.043000,-2.749000,-8.334000,0.07700,-1.419000,0.696000,1.677000,-3.706000,-0.5140,-6.264000,0.108104,-6.84000,0.136578,-0.944000,-7.018000,0.71800,-7.524000,-2.764000,-2.833000,-6.06100,-6.00600,-0.890000,-2.897000,-4.041000,-0.985000,-0.904000,-1.080000,-0.774000,-0.589000,-1.118000,-0.738000,-1.258000,-0.782000,-0.466000,-15.556000,-0.029000,-0.014000,-0.010000,-1.733000,-2.840000,-0.993000,-1.66600,-0.961000,-3.000000,-3.620000,-0.544000,-0.916000,-0.975000,-0.726000,0.771000,-1.317000,-3.230000,-9.909000,-10.692000,-2.392000,-3.45900,-3.28500,0.500000,0.830000,1


In [0]:
df1 = data.iloc[:1351, :]
df2 = data.iloc[1351:, :]

df_1 = data.iloc[:1351, 1:-1]
df_2 = data.iloc[1351:,1:-1]

df_1.values

array([[ 8.39800e+02,  4.78000e+02,  6.16400e+00, ..., -2.00000e+00,
        -2.50000e-01, -5.00000e-01],
       [ 1.05600e+02,  2.04000e+02,  0.00000e+00, ..., -1.39200e+00,
         0.00000e+00, -1.60000e-01],
       [ 1.68609e+03,  2.01000e+03,  7.89500e+00, ...,  3.72800e+00,
        -7.50000e-01, -8.30000e-01],
       ...,
       [ 4.63340e+02,  4.94000e+02,  3.24000e+00, ...,  2.42600e+00,
        -5.00000e-01, -5.00000e-01],
       [ 3.29440e+02,  3.54000e+02,  3.16200e+00, ...,  1.32200e+00,
         0.00000e+00, -6.70000e-01],
       [-1.53910e+02, -4.34000e+02, -2.33000e+00, ..., -8.30000e-01,
         0.00000e+00,  6.60000e-01]])

In [0]:
tensor=[]

for i in range(df_1.shape[0]):
    #df_1.iloc[i] = torch.tensor(df_1[:], dtype=torch.float)
    b= torch.from_numpy(df_1.iloc[i].values)
    #tensor.view(1, 65)
    tensor.append(b)

NameError: ignored

In [0]:
len(tensor)

for i in range(0, len(tensor)):
  tensor[i]= tensor[i].view(1,65)
  
tensor[1].size

<function Tensor.size>

In [0]:
tensor[0]

tensor([[ 8.3980e+02,  4.7800e+02,  6.1640e+00, -2.9100e-01,  3.8690e+00,
          1.4740e+00,  4.0000e-03,  1.9460e+00,  5.0910e+00, -1.1300e-01,
          2.1930e+00, -4.3200e-01, -1.0300e+00,  2.6760e+00,  6.4600e-01,
          3.7040e+00, -8.7896e-02,  4.0230e+00, -1.1342e-01,  2.4640e+00,
          2.0880e+00, -6.3500e-01,  1.2230e+00,  2.3030e+00,  2.7840e+00,
          4.1540e+00,  3.8780e+00,  1.0380e+00,  1.9130e+00,  1.7590e+00,
          1.1490e+00,  8.0200e-01,  1.0120e+00,  9.2700e-01,  1.5720e+00,
          8.0100e-01,  1.0430e+00,  9.7300e-01,  9.5400e-01,  2.0430e+00,
          9.3330e+00,  7.7000e-02,  4.0000e-02,  8.0000e-03,  3.5280e+00,
          2.9870e+00,  1.4810e+00,  2.8210e+00,  2.2790e+00,  2.4090e+00,
          3.3610e+00,  5.3700e-01,  5.2100e-01,  6.2900e-01,  1.3910e+00,
         -3.2800e-01,  1.9350e+00,  2.8990e+00,  1.3969e+01,  1.3591e+01,
          2.8550e+00,  2.8990e+00, -2.0000e+00, -2.5000e-01, -5.0000e-01]],
       dtype=torch.float64)

In [0]:
import logging
import torch.nn as nn
import numpy as np


class BaseNet(nn.Module):
    """Base class for all neural networks."""

    def __init__(self):
        super().__init__()
        self.logger = logging.getLogger(self.__class__.__name__)
        self.rep_dim = None  # representation dimensionality, i.e. dim of the last layer

    def forward(self, *input):
        """
        Forward pass logic
        :return: Network output
        """
        raise NotImplementedError

    def summary(self):
        """Network summary."""
        net_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in net_parameters])
        self.logger.info('Trainable parameters: {}'.format(params))
        self.logger.info(self)
        


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from base.base_net import BaseNet


class Net(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 10
        self.pool = nn.MaxPool2d(1, 2)

        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        #self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        #self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(65, self.rep_dim, bias=False)
        self.fc2 = nn.Linear(self.rep_dim, 2, bias=False)
        

    def forward(self, x):
        x = F.relu((self.fc1(input)))
        x = F.softmax(self.fc2(x), dim=1)
        #x = self.conv1(x)
        #x = self.pool(F.leaky_relu(self.fc1(x)))
        #x = self.conv2(x)
        #x = self.pool(F.leaky_relu(self.fc2(x)))
        #x = x.view(x.size(0), -1)
        #x = self.fc1(x)
        return x
net = Net()
print(net)




class MNIST_LeNet_Autoencoder(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 63
        self.pool = nn.MaxPool2d(2, 2)

        # Encoder (must match the Deep SVDD network above)
        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 4 * 7, self.rep_dim, bias=False)

        # Decoder
        self.conv3 = nn.Conv2d(2, 4, 5, bias=False, padding=2)
        self.bn3 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.conv4 = nn.Conv2d(4, 8, 5, bias=False, padding=1)
        self.bn4 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv5 = nn.Conv2d(8, 1, 5, bias=False, padding=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = x.view(x.size(0), int(self.rep_dim / 16), 4, 4)
        x = F.interpolate(F.leaky_relu(x), scale_factor=2)
        x = self.conv3(x)
        x = F.interpolate(F.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.conv4(x)
        x = F.interpolate(F.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.conv5(x)
        x = torch.sigmoid(x)
        return x

Net(
  (pool): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (conv2): Conv2d(8, 4, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  (fc1): Linear(in_features=65, out_features=10, bias=False)
  (fc2): Linear(in_features=10, out_features=2, bias=False)
)


In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

3
torch.Size([8, 1, 5, 5])


In [0]:
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        self.linear1 = nn.Linear(in_features=65, out_features=32)
        #self.bn1 = nn.BatchNorm1d(num_features=320)
        self.linear2 = nn.Linear(in_features=32, out_features=2)

    def forward(self, input):  # Input is a 1D tensor
        y = F.relu((self.linear1(input)))
        y = F.softmax(self.linear2(y), dim=1)
        return y
    
model = network()
x = torch.from_numpy(df_1.iloc[i].values).view(1,65).float()
#torch.from_numpy(df_1.iloc[i].values).view(1,65)
output = model(x)

In [0]:
input = torch.from_numpy(df_1.iloc[i].values).view(1,65).float()
out = net(input)
print(out)

tensor([[1.0000e+00, 1.4420e-07]], grad_fn=<SoftmaxBackward>)


In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

NameError: ignored

In [0]:
class Autoencoder:

    def __init__(self, dataset):
        super().__init__()

        self.rep_dim = 65
        self.pool = nn.MaxPool2d(2, 2)

        # Encoder (must match the Deep SVDD network above)
        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5,  bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 7 , self.rep_dim, bias=False)

        # Decoder
        self.deconv1 = nn.ConvTranspose2d(2, 4, 5,  bias=False, padding=2)
        self.bn3 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(4, 8, 5,  bias=False, padding=3)
        self.bn4 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(8, 1, 5, bias=False, padding=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = x.view(x.size(0), int(self.rep_dim / 16), 4, 4)
        x = F.interpolate(F.leaky_relu(x), scale_factor=2)
        x = self.deconv1(x)
        x = F.interpolate(F.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.deconv3(x)
        x = torch.sigmoid(x)

        return x

In [0]:
def build_autoencoder(net_name):
    """Builds the corresponding autoencoder network."""

    implemented_networks = ('mnist_LeNet', 'dataset')
    assert net_name in implemented_networks

    ae_net = None

    if net_name == 'mnist_LeNet':
        ae_net = MNIST_LeNet_Autoencoder()

    if net_name == 'dataset':
        ae_net = Autoencoder()

    return ae_net


In [0]:
import json
import torch

from base.base_dataset import BaseADDataset
from networks.main import build_network, build_autoencoder
from optim.deepSVDD_trainer import DeepSVDDTrainer
from optim.ae_trainer import AETrainer

dataset = tensor 

class DeepSVDD:
    """A class for the Deep SVDD method.
    Attributes:
        objective: A string specifying the Deep SVDD objective (either 'one-class' or 'soft-boundary').
        nu: Deep SVDD hyperparameter nu (must be 0 < nu <= 1).
        R: Hypersphere radius R.
        c: Hypersphere center c.
        net_name: A string indicating the name of the neural network to use.
        net: The neural network \phi.
        ae_net: The autoencoder network corresponding to \phi for network weights pretraining.
        trainer: DeepSVDDTrainer to train a Deep SVDD model.
        optimizer_name: A string indicating the optimizer to use for training the Deep SVDD network.
        ae_trainer: AETrainer to train an autoencoder in pretraining.
        ae_optimizer_name: A string indicating the optimizer to use for pretraining the autoencoder.
        results: A dictionary to save the results.
    """

    def __init__(self, dataset, objective: str = 'one-class', nu: float = 0.1):
        """Inits DeepSVDD with one of the two objectives and hyperparameter nu."""

        assert objective in ('one-class', 'soft-boundary'), "Objective must be either 'one-class' or 'soft-boundary'."
        self.objective = objective
        assert (0 < nu) & (nu <= 1), "For hyperparameter nu, it must hold: 0 < nu <= 1."
        self.nu = nu
        self.R = 0.0  # hypersphere radius R
        self.c = None  # hypersphere center c

        self.net_name = None
        self.net = None  # neural network \phi

        self.trainer = None
        self.optimizer_name = None

        self.ae_net = None  # autoencoder network for pretraining
        self.ae_trainer = None
        self.ae_optimizer_name = None

        self.results = {
            'train_time': None,
            'test_auc': None,
            'test_time': None,
            'test_scores': None,
        }

    def set_network(self, net_name):
        """Builds the neural network \phi."""
        self.net_name = net_name
        self.net = build_network(net_name)

    def train(self, dataset: BaseADDataset, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 50,
              lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
              n_jobs_dataloader: int = 0):
        """Trains the Deep SVDD model on the training data."""

        self.optimizer_name = optimizer_name
        self.trainer = DeepSVDDTrainer(self.objective, self.R, self.c, self.nu, optimizer_name, lr=lr,
                                       n_epochs=n_epochs, lr_milestones=lr_milestones, batch_size=batch_size,
                                       weight_decay=weight_decay, device=device, n_jobs_dataloader=n_jobs_dataloader)
        # Get the model
        self.net = self.trainer.train(dataset, self.net)
        self.R = float(self.trainer.R.cpu().data.numpy())  # get float
        self.c = self.trainer.c.cpu().data.numpy().tolist()  # get list
        self.results['train_time'] = self.trainer.train_time

    def test(self, dataset: BaseADDataset, device: str = 'cuda', n_jobs_dataloader: int = 0):
        """Tests the Deep SVDD model on the test data."""

        if self.trainer is None:
            self.trainer = DeepSVDDTrainer(self.objective, self.R, self.c, self.nu,
                                           device=device, n_jobs_dataloader=n_jobs_dataloader)

        self.trainer.test(dataset, self.net)
        # Get results
        self.results['test_auc'] = self.trainer.test_auc
        self.results['test_time'] = self.trainer.test_time
        self.results['test_scores'] = self.trainer.test_scores

    def pretrain(self, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 100,
                 lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        """Pretrains the weights for the Deep SVDD network \phi via autoencoder."""

        self.ae_net = build_autoencoder(self.net_name)
        self.ae_optimizer_name = optimizer_name
        self.ae_trainer = AETrainer(optimizer_name, lr=lr, n_epochs=n_epochs, lr_milestones=lr_milestones,
                                    batch_size=batch_size, weight_decay=weight_decay, device=device,
                                    n_jobs_dataloader=n_jobs_dataloader)
        self.ae_net = self.ae_trainer.train(dataset, self.ae_net)
        self.ae_trainer.test(dataset, self.ae_net)
        self.init_network_weights_from_pretraining()

    def init_network_weights_from_pretraining(self):
        """Initialize the Deep SVDD network weights from the encoder weights of the pretraining autoencoder."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()

        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)
        # Load the new state_dict
        self.net.load_state_dict(net_dict)

    def save_model(self, export_model, save_ae=True):
        """Save Deep SVDD model to export_model."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict() if save_ae else None

        torch.save({'R': self.R,
                    'c': self.c,
                    'net_dict': net_dict,
                    'ae_net_dict': ae_net_dict}, export_model)

    def load_model(self, model_path, load_ae=False):
        """Load Deep SVDD model from model_path."""

        model_dict = torch.load(model_path)

        self.R = model_dict['R']
        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])
        if load_ae:
            if self.ae_net is None:
                self.ae_net = build_autoencoder(self.net_name)
            self.ae_net.load_state_dict(model_dict['ae_net_dict'])

    def save_results(self, export_json):
        """Save results dict to a JSON-file."""
        with open(export_json, 'w') as fp:
            json.dump(self.results, fp)

In [0]:
DeepSVDD.pretrain(tensor)

AttributeError: ignored